In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)

import matplotlib.pyplot as plt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# %matplotlib inline
PROJECT_ROOT = Path.cwd().parent
data_path = PROJECT_ROOT.joinpath('data')
print(PROJECT_ROOT)

/home/gabe/Coding/nlpcourseproject


In [11]:
data = pd.read_csv(data_path.joinpath('accuracy_run_results.csv'))

medians = data.groupby(['prompt_id','prompt_name','prompt_task']).median().reset_index()

In [109]:

task_colors = {'CLASSIFICATION': '#1f77b4',
 'COMPLETION': '#ff7f0e',
 'ENTAILMENT': '#2ca02c',
 'MCQ': '#d62728',
 'QA': '#9467bd',
 'SENTIMENT': '#8c564b',
 'SUMMARIZATION': '#e377c2'}

GROUP_NAME_MAPPING = {
    "anli[dev_r1]"                            : "ANLI R1",
    "anli[dev_r2]"                            : "ANLI R2",
    "anli[dev_r3]"                            : "ANLI R3",
    "CommitmentBank[validation]"              : "CB",
    "AQuA[validation]"                        : "AQuA",
    "WordsinContext[validation]"              : "WiC",
    "RecognizingTextualEntailment[validation]": "RTE",
    "craigslist_bargains[validation]"         : "Craigslist",
}
valid_metrics = ['f1', 'em', 'accuracy']

# subplot_count = 

figure_object = {}

inverse_map_names = {v:k for k,v in GROUP_NAME_MAPPING.items()}
subplot_titles = sorted(list(inverse_map_names))


fig = make_subplots(
    1,
    2,
    subplot_titles=["Accuracy","F1"],
)
f1_chart = go.Scatter(
    x=medians['prompt_tokens'].tolist(),
    y=medians['f1_rank'].tolist(),
    mode='markers',
    showlegend=False
)
fig.append_trace(f1_chart, row=1, col=2)

acc_chart = go.Scatter(
    x=medians['prompt_tokens'].tolist(),
    y=medians['accuracy_rank'].tolist(),
    mode='markers',
    showlegend=False
    
)
fig.append_trace(acc_chart, row=1, col=1)
fig.update_layout(
#                 title=title+" - "+met_name,
    title_x=0.5,
    font=dict(size=15),
    template="plotly_white",
    legend_orientation='h',
    legend=dict(xanchor="center", x=0.5, bgcolor="rgba(0,0,0,0)"),
    # yaxis=dict(range=[0,100]),
    width=1000,
    height=600,
)
fig['layout']['xaxis']['title']='Length of Prompt (# of Tokens)'
fig['layout']['xaxis2']['title']='Length of Prompt (# of Tokens)'
fig['layout']['yaxis']['autorange'] = "reversed"
fig['layout']['yaxis2']['autorange'] = "reversed"
fig['layout']['yaxis']['title']='Median Rank out of 98 Prompts'
fig['layout']['yaxis2']['title']='Median Rank out of 98 Prompts'
fig.update_yaxes(range=[0,100])# hide all the xticks
fig.write_image(re.sub(' ', '-', 'ranks_graphs.png'))
fig.show()


In [10]:
medians.sort_values(by=['accuracy_rank'])[["prompt_name",'prompt_task','accuracy_rank','f1_rank','f1','accuracy','prompt_tokens']]

,prompt_name,prompt_task,accuracy_rank,f1_rank,f1,accuracy,prompt_tokens
88,must_be_true,ANLI,9.75,9.00,26.231477,34.575000,14.0
34,generate_rational_and_correct_choice,AQuA,12.75,15.00,23.992104,36.057956,33.0
57,based_on_the_previous_passage,ANLI,14.00,12.00,26.759136,36.250000,15.0
7,select_the_best_option,AQuA,14.00,11.50,26.328158,35.565476,12.0
62,more_likely,COPA,16.00,13.00,25.478826,34.300000,19.0
70,Answer_questions_from_options,AQuA,16.25,34.50,25.335470,35.297619,11.0
82,gre_problem,MathQA,16.75,11.00,24.657357,34.841667,8.0
35,generate_rationale,AQuA,16.75,12.50,26.184389,34.300000,17.0
58,does_this_imply,ANLI,17.25,24.50,24.180612,34.100000,23.0
12,fill_in_the_blank_with_choices_after,NumerSense,17.50,21.50,23.280519,33.715410,20.0


In [12]:
data['rum_name']=='CTNoText'

KeyError: 'rum_name'